In [44]:
import pandas as pd
from sqlalchemy import create_engine

# SQFT Dataset

In [45]:
sqft_raw = pd.read_csv('Raw_Data/City_MedianValuePerSqft_AllHomes.csv', encoding = 'latin-1')
sqft_raw.head()

,RegionID,RegionName,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12
0,6181,New York,NY,New York-Newark-Jersey City,Queens County,1,130.0,131.0,131.0,132.0,...,506,510,514,518,519,520,522,525,528,529
1,12447,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles County,2,111.0,111.0,111.0,110.0,...,454,455,457,458,459,462,464,466,468,469
2,17426,Chicago,IL,Chicago-Naperville-Elgin,Cook County,3,87.0,88.0,89.0,89.0,...,169,170,171,171,171,171,172,173,173,173
3,39051,Houston,TX,Houston-The Woodlands-Sugar Land,Harris County,4,50.0,50.0,50.0,51.0,...,101,102,103,103,103,104,104,104,105,106
4,6915,San Antonio,TX,San Antonio-New Braunfels,Bexar County,5,54.0,54.0,53.0,53.0,...,103,104,105,105,106,106,107,108,109,109


In [46]:
# sqft_raw.Metro.loc[sqft_raw.State == 'WA']

In [47]:
# Grab only rows for specified metropolitan areas
cities = ['Seattle', 'Washington', 'Detroit', 'Denver', 'Austin', 'San Francisco', 
          'Dallas', 'New York', 'Orlando', 'Raleigh', 'Durham']
metro_index = []
for city in cities:
    metro_index.append(sqft_raw.index[sqft_raw['Metro'].str.contains(city, na = False)].tolist())
    


metro_index = [item for sublist in metro_index for item in sublist]

sqft_metro = sqft_raw.iloc[metro_index]
sqft_metro.head()

,RegionID,RegionName,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12
23,16037,Seattle,WA,Seattle-Tacoma-Bellevue,King County,24,113.0,113.0,112.0,113.0,...,506,506,507,508,505,502,502,500,497,497
129,27362,Tacoma,WA,Seattle-Tacoma-Bellevue,Pierce County,130,84.0,84.0,85.0,84.0,...,226,228,230,232,234,235,236,238,241,244
217,13480,Renton,WA,Seattle-Tacoma-Bellevue,King County,218,89.0,88.0,89.0,88.0,...,256,257,258,259,259,258,258,257,256,256
232,3619,Bellevue,WA,Seattle-Tacoma-Bellevue,King County,233,106.0,106.0,105.0,105.0,...,446,446,449,450,449,448,449,448,446,446
326,5399,Kent,WA,Seattle-Tacoma-Bellevue,King County,327,85.0,85.0,84.0,84.0,...,213,212,213,213,213,212,213,213,212,212


In [48]:
# Set up Raleigh-Durham-Chapel Hill area
sqft_metro['Metro'] = sqft_metro['Metro'].replace(['Durham-Chapel Hill', 'Raleigh'], 'Raleigh-Durham-Chapel Hill')

C:\Users\Will\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [49]:
#Group regions under each metro area
metro_group = sqft_metro.groupby('Metro').mean()
metro_group.drop(index = ['Washington Court House', 'Austin'], columns = ['RegionID', 'SizeRank'], inplace = True)

# Transpose dataset
metro_group = metro_group.transpose().reset_index()
metro_group = metro_group.rename(columns = {'index': 'Month'})
metro_group.head()

Metro,Month,Austin-Round Rock,Dallas-Fort Worth-Arlington,Denver-Aurora-Lakewood,Detroit-Warren-Dearborn,New York-Newark-Jersey City,Orlando-Kissimmee-Sanford,Raleigh-Durham-Chapel Hill,San Francisco-Oakland-Hayward,Seattle-Tacoma-Bellevue,Washington-Arlington-Alexandria
0,1996-04,81.810811,61.217391,91.016949,81.773196,109.122164,55.982759,60.710526,174.650602,95.105263,93.542553
1,1996-05,81.837838,61.268116,91.135593,82.206186,109.148342,56.051724,60.710526,174.710843,95.052632,93.521277
2,1996-06,81.837838,61.304348,91.406780,82.793814,109.101222,56.120690,60.842105,174.614458,94.978947,93.329787
3,1996-07,81.675676,61.463768,91.627119,83.340206,108.945899,55.965517,60.921053,174.686747,94.884211,93.063830
4,1996-08,81.567568,61.565217,91.813559,83.701031,108.767888,56.000000,60.921053,174.759036,94.936842,92.760638


# Sales Count Dataset

In [50]:
us_state_abbrev = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
    'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO',
    'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH',
    'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT',
    'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY',
    'District of Columbia': 'DC'
}

In [51]:
sales_raw = pd.read_csv('Raw_Data/Sale_Counts_Seas_Adj_City.csv', encoding = 'latin-1')
sales_raw.head()

,RegionID,RegionName,StateName,SizeRank,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,...,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12
0,12447.0,Los Angeles,California,1,1626.0,1626.0,1593.0,1694.0,1768.0,1780.0,...,2913.0,2836.0,2791.0,2747.0,2678.0,2638.0,NaN,NaN,NaN,NaN
1,17426.0,Chicago,Illinois,2,3816.0,3678.0,3137.0,2778.0,2789.0,2745.0,...,4044.0,4109.0,3922.0,3775.0,3645.0,3311.0,2973.0,2833.0,2975.0,3316.0
2,39051.0,Houston,Texas,3,NaN,NaN,NaN,NaN,NaN,NaN,...,3391.0,3324.0,3329.0,3365.0,3320.0,3368.0,3329.0,3428.0,3383.0,3365.0
3,6915.0,San Antonio,Texas,4,1491.0,1408.0,1303.0,1325.0,1288.0,1258.0,...,2141.0,2023.0,1893.0,1795.0,1762.0,1763.0,1753.0,1811.0,1874.0,1874.0
4,13271.0,Philadelphia,Pennsylvania,5,2005.0,1898.0,1748.0,1685.0,1647.0,1613.0,...,2418.0,2341.0,2351.0,2314.0,2255.0,2143.0,2168.0,2171.0,2208.0,2351.0


In [52]:
# Change state names to abbreviations
sales_raw = sales_raw.replace({'StateName' : us_state_abbrev})

In [53]:
# Group regions into their metropolitan areas
sales_raw['RegionID'] = sales_raw['RegionID'].astype(int).astype(str)
sqft_metro['RegionID'] = sqft_metro['RegionID'].astype(str)

sales = list(sqft_metro['RegionID'])

sales_index = []
for sale in sales:
  if len(sales_raw.loc[sales_raw['RegionID'] == sale].index) > 0:
    sales_index.append(sales_raw.loc[sales_raw['RegionID'] == sale].index[0])

sales_metro = sales_raw.iloc[sales_index]

C:\Users\Will\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [54]:
sales_metro.head()

,RegionID,RegionName,StateName,SizeRank,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,...,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12
22,16037,Seattle,WA,23,876.0,745.0,589.0,557.0,544.0,517.0,...,964.0,892.0,852.0,823.0,821.0,778.0,736.0,722.0,723.0,738.0
109,27362,Tacoma,WA,110,202.0,198.0,185.0,196.0,189.0,176.0,...,404.0,390.0,394.0,406.0,408.0,378.0,354.0,357.0,370.0,362.0
170,13480,Renton,WA,171,132.0,124.0,109.0,102.0,96.0,81.0,...,207.0,207.0,184.0,198.0,195.0,186.0,167.0,153.0,161.0,162.0
184,3619,Bellevue,WA,185,NaN,NaN,NaN,NaN,NaN,NaN,...,198.0,189.0,176.0,186.0,177.0,166.0,142.0,133.0,141.0,142.0
250,5399,Kent,WA,251,125.0,105.0,87.0,77.0,80.0,76.0,...,171.0,156.0,149.0,150.0,164.0,155.0,142.0,128.0,136.0,141.0


In [55]:
# Generate list of metro area names
metro_names = list(sqft_metro.Metro.unique())
metro_names.remove('Washington Court House')
metro_names.remove('Austin')

metro_names = ['Seattle-Tacoma-Bellevue',
 'Washington-Arlington-Alexandria',
 'Detroit-Warren-Dearborn',
 'Denver-Aurora-Lakewood',
 'Austin-Round Rock',
 'Orlando-Kissimmee-Sanford',
 'Raleigh-Durham-Chapel Hill', 'Dallas-Fort Worth-Arlington', 'San Francisco-Oakland-Hayward', 'New York-Newark-Jersey City']

In [56]:
# Dictionary of metro aread and states
metro_agg = []
for i in range(0, len(metro_names)):
    state = list(sqft_metro.loc[sqft_metro.Metro == metro_names[i]].State.unique())
    metro_agg.append({metro_names[i] : state})

metro_agg

[{'Seattle-Tacoma-Bellevue': ['WA']},
 {'Washington-Arlington-Alexandria': ['DC', 'MD', 'VA', 'WV']},
 {'Detroit-Warren-Dearborn': ['MI']},
 {'Denver-Aurora-Lakewood': ['CO']},
 {'Austin-Round Rock': ['TX']},
 {'Orlando-Kissimmee-Sanford': ['FL']},
 {'Raleigh-Durham-Chapel Hill': ['NC']},
 {'Dallas-Fort Worth-Arlington': ['TX']},
 {'San Francisco-Oakland-Hayward': ['CA']},
 {'New York-Newark-Jersey City': ['NY', 'NJ']}]

In [57]:
sales_agg = pd.DataFrame()
for i in range(0, len(metro_names)):
    for j in range(0, len(metro_agg[i][metro_names[i]])):
        WA_sales = sales_metro.loc[sales_metro.StateName == metro_agg[i][metro_names[i]][j]]
        WA_metro = sqft_metro.loc[sqft_metro.State == metro_agg[i][metro_names[i]][j]]
        WA_cities = [value for value in WA_sales.RegionName.unique() if value in WA_metro.RegionName.unique()]

        # RegionID for every city in Seattle metro area
        WA_agg = WA_sales.loc[sales_metro['RegionName'].isin(WA_cities)]
        WA_agg['Metro'] = metro_names[i]
        sales_agg = sales_agg.append(WA_agg, ignore_index = True)
sales_agg.head()

,RegionID,RegionName,StateName,SizeRank,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,...,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,Metro
0,16037,Seattle,WA,23,876.0,745.0,589.0,557.0,544.0,517.0,...,892.0,852.0,823.0,821.0,778.0,736.0,722.0,723.0,738.0,Seattle-Tacoma-Bellevue
1,27362,Tacoma,WA,110,202.0,198.0,185.0,196.0,189.0,176.0,...,390.0,394.0,406.0,408.0,378.0,354.0,357.0,370.0,362.0,Seattle-Tacoma-Bellevue
2,13480,Renton,WA,171,132.0,124.0,109.0,102.0,96.0,81.0,...,207.0,184.0,198.0,195.0,186.0,167.0,153.0,161.0,162.0,Seattle-Tacoma-Bellevue
3,3619,Bellevue,WA,185,NaN,NaN,NaN,NaN,NaN,NaN,...,189.0,176.0,186.0,177.0,166.0,142.0,133.0,141.0,142.0,Seattle-Tacoma-Bellevue
4,5399,Kent,WA,251,125.0,105.0,87.0,77.0,80.0,76.0,...,156.0,149.0,150.0,164.0,155.0,142.0,128.0,136.0,141.0,Seattle-Tacoma-Bellevue


In [58]:
sales_group_mean = sales_agg.groupby('Metro').mean()
sales_group_mean.drop(columns = ['SizeRank'], inplace = True)

# Transpose dataset
sales_group_mean = sales_group_mean.transpose().reset_index()
sales_group_mean = sales_group_mean.rename(columns = {'index': 'Month'})
sales_group_mean.head()

Metro,Month,Austin-Round Rock,Dallas-Fort Worth-Arlington,Denver-Aurora-Lakewood,Detroit-Warren-Dearborn,New York-Newark-Jersey City,Orlando-Kissimmee-Sanford,Raleigh-Durham-Chapel Hill,San Francisco-Oakland-Hayward,Seattle-Tacoma-Bellevue,Washington-Arlington-Alexandria
0,2008-03,61.201550,61.201550,94.295455,30.155844,14.467562,33.113636,84.2500,46.366667,43.641026,25.401575
1,2008-04,56.976744,56.976744,84.590909,31.233766,14.324385,30.318182,79.7500,44.583333,39.807692,24.606299
2,2008-05,52.348837,52.348837,77.431818,30.454545,13.796421,28.750000,69.4375,39.800000,34.115385,23.141732
3,2008-06,52.689922,52.689922,76.113636,28.974026,13.185682,28.318182,66.1250,40.250000,32.282051,22.937008
4,2008-07,52.000000,52.000000,76.704545,28.454545,12.870246,30.068182,65.3125,42.180328,30.448718,23.086614


In [59]:
sales_group_median = sales_agg.groupby('Metro').median()
sales_group_median.drop(columns = ['SizeRank'], inplace = True)

# Transpose dataset
sales_group_median = sales_group_median.transpose().reset_index()
sales_group_median = sales_group_median.rename(columns = {'index': 'Month'})
sales_group_median.head()

Metro,Month,Austin-Round Rock,Dallas-Fort Worth-Arlington,Denver-Aurora-Lakewood,Detroit-Warren-Dearborn,New York-Newark-Jersey City,Orlando-Kissimmee-Sanford,Raleigh-Durham-Chapel Hill,San Francisco-Oakland-Hayward,Seattle-Tacoma-Bellevue,Washington-Arlington-Alexandria
0,2008-03,23.0,23.0,23.0,18.0,9.0,10.0,22.0,26.0,20.5,11.0
1,2008-04,21.0,21.0,25.5,20.0,9.0,9.5,20.5,22.0,17.5,10.0
2,2008-05,17.0,17.0,21.0,19.0,9.0,9.0,20.0,18.5,17.0,10.0
3,2008-06,17.0,17.0,21.5,17.0,9.0,8.5,18.0,20.5,16.0,9.0
4,2008-07,16.0,16.0,20.0,15.0,8.0,11.0,15.0,23.0,14.5,9.0


# Median Sale Prices Dataset

In [60]:
price_raw = pd.read_csv('Raw_Data/Sale_Prices_Zip.csv', encoding = 'latin-1')
price_raw.rename(columns = {'RegionName':'ZipCode'}, inplace = True)
price_raw.head()

,RegionID,ZipCode,StateName,SizeRank,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,...,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12
0,61639.0,10025,New York,1,NaN,NaN,NaN,NaN,NaN,NaN,...,885100.0,1076100.0,1086300.0,1053100.0,952800.0,1007400.0,1108000.0,1159900.0,1083400.0,968600.0
1,84654.0,60657,Illinois,2,NaN,NaN,NaN,NaN,NaN,NaN,...,381600.0,340200.0,334100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,91982.0,77494,Texas,3,NaN,NaN,NaN,NaN,NaN,NaN,...,336100.0,333700.0,330600.0,333900.0,322300.0,321400.0,309200.0,311300.0,304300.0,NaN
3,93144.0,79936,Texas,4,NaN,NaN,NaN,NaN,NaN,NaN,...,128100.0,126500.0,126700.0,131000.0,131900.0,130800.0,128900.0,130100.0,128400.0,126900.0
4,91940.0,77449,Texas,5,NaN,NaN,NaN,NaN,NaN,NaN,...,180100.0,182100.0,178100.0,178600.0,178700.0,180800.0,184500.0,187400.0,191000.0,190700.0


### Aggregate list of zip codes by metro area

In [61]:
zip_raw = pd.read_csv('Raw_Data/Zip_Zhvi_AllHomes.csv', encoding = 'latin-1')
zip_raw.rename(columns = {'RegionName':'ZipCode'}, inplace = True)
zip_raw.head()

,RegionID,ZipCode,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12
0,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,171600.0,171600.0,171400.0,...,1122300,1117600,1115600,1110000,1098000,1086700,1080400,1072600,1063400,1058600
1,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2,158400.0,159700.0,160700.0,...,352700,351400,349900,348300,347200,347400,348800,350900,352700,353500
2,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,3,347900.0,349600.0,351100.0,...,1496600,1494700,1502400,1489400,1462300,1438000,1411200,1390300,1378900,1370500
3,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,4,210400.0,212200.0,212200.0,...,330400,332700,334500,335900,337000,338300,338400,336900,336000,336500
4,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,5,192500.0,194500.0,196100.0,...,430100,428800,427900,428200,429900,430900,430000,430200,431500,432300


In [62]:
# filter zip codes by metro area
zip_metro = zip_raw.loc[zip_raw['Metro'].isin(metro_names)]

# Create Realeigh-Durham-Chapel Hill area
zip_NC = zip_raw.loc[zip_raw['Metro'].isin(['Raleigh', 'Durham-Chapel Hill'])]
zip_NC['Metro'] = 'Raleigh-Durham-Chapel Hill'
zip_metro = zip_metro.append(zip_NC)

# Create dictionary of zip codes by metro area
zip_agg = []
for i in range(0, len(metro_names)):
    zip_agg.append({metro_names[i]:list((zip_metro.loc[zip_metro['Metro'] == metro_names[i]].ZipCode).values)})

C:\Users\Will\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [63]:
price_agg = pd.DataFrame()

# metro area, all zips, single zip
for i in range(0, len(metro_names)):
    zipper = zip_agg[i][metro_names[i]]

    metro_zips = price_raw.loc[price_raw['ZipCode'].isin(zipper)]
    metro_zips['Metro'] = metro_names[i]
    price_agg = price_agg.append(metro_zips)
    
price_agg.head()

C:\Users\Will\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,RegionID,ZipCode,StateName,SizeRank,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,...,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,Metro
38,99534.0,98052,Washington,39,429000.0,428800.0,419600.0,414600.0,410600.0,453200.0,...,759500.0,771100.0,773200.0,743000.0,751500.0,785400.0,822100.0,808100.0,782600.0,Seattle-Tacoma-Bellevue
93,99500.0,98012,Washington,94,385400.0,375800.0,366600.0,374700.0,367100.0,371300.0,...,581000.0,588700.0,589700.0,589900.0,578400.0,557200.0,536000.0,539200.0,551300.0,Seattle-Tacoma-Bellevue
117,99563.0,98103,Washington,118,NaN,NaN,NaN,NaN,NaN,NaN,...,776600.0,804200.0,795400.0,755400.0,740500.0,768300.0,805700.0,775000.0,732200.0,Seattle-Tacoma-Bellevue
249,99589.0,98133,Washington,250,318000.0,338800.0,321400.0,336500.0,331600.0,324200.0,...,544200.0,531700.0,558300.0,565000.0,550800.0,551600.0,546300.0,572400.0,560400.0,Seattle-Tacoma-Bellevue
261,99581.0,98122,Washington,262,NaN,NaN,NaN,NaN,NaN,NaN,...,768600.0,762400.0,754800.0,753300.0,788900.0,760200.0,728100.0,717300.0,684400.0,Seattle-Tacoma-Bellevue


# Create Database Connection

In [64]:
# engine = create_engine('sqlite:///real_estate.sqlite')

In [65]:
# engine.table_names()

In [66]:
# metro_group.to_sql(
#     name = 'median_price_sqft', con = engine,
#     if_exists = 'replace')

In [67]:
# sales_group_median.to_sql(
#     name = 'median_sales_count', con = engine,
#     if_exists = 'replace')

In [68]:
# sales_group_mean.to_sql(
#     name = 'mean_sales_count', con = engine,
#     if_exists = 'replace')

In [69]:
# price_agg.to_sql(
#     name = 'median_price_zip', con = engine,
#     if_exists = 'replace')

In [70]:
# pd.read_sql('SELECT * FROM median_price_zip', con = engine)

# MySQL Connection

In [71]:
import pymysql
pymysql.install_as_MySQLdb()

In [72]:
# Define database within MySQL client

In [74]:
connection_string = (f"root:eAVuM6bo4LgR@localhost/")

engine = create_engine(f"mysql://{connection_string}")
engine.execute("DROP DATABASE IF EXISTS real_estate")
engine.execute("CREATE DATABASE real_estate")

C:\Users\Will\Anaconda3\envs\PythonData\lib\site-packages\pymysql\cursors.py:170: Warning: (1008, "Can't drop database 'real_estate'; database doesn't exist")
  result = self._query(query)


In [76]:
engine.execute("USE real_estate") # select new db
sales_group_mean.to_sql(
    name = 'mean_sales_count', con = engine,
    if_exists = 'replace')
with engine.connect() as con:
    con.execute('ALTER TABLE `mean_sales_count` ADD PRIMARY KEY (`index`);')

In [77]:
engine.execute("USE real_estate")
price_agg.to_sql(
    name = 'median_price_zip', con = engine,
    if_exists = 'replace', chunksize = 75)
with engine.connect() as con:
    con.execute('ALTER TABLE `median_price_zip` ADD PRIMARY KEY (`ZipCode`);')

In [78]:
engine.execute("USE real_estate") # select new db
sales_group_median.to_sql(
    name = 'median_sales_count', con = engine,
    if_exists = 'replace')
with engine.connect() as con:
    con.execute('ALTER TABLE `median_sales_count` ADD PRIMARY KEY (`index`);')

In [79]:
engine.execute("USE real_estate") # select new db
metro_group.to_sql(
    name = 'median_price_sqft', con = engine,
    if_exists = 'replace')
with engine.connect() as con:
    con.execute('ALTER TABLE `median_price_sqft` ADD PRIMARY KEY (`index`);')